In [1]:
import numpy as np
import scipy.io
from scipy.signal import correlate
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Carregando os dados do arquivo data_TRMS.mat
data = scipy.io.loadmat('data_TRMS.mat')
t = data['t'].flatten()
u = data['u'].flatten()
y = data['y'].flatten()

# 1. Pré-processamento
# a) Calculando as Funções de Autocorrelação (FAC)
tau_max = 100  # Número máximo de atrasos a serem considerados
fac_y = correlate(y, y, mode='full')
fac_y2 = correlate(y ** 2, y ** 2, mode='full')
tau_y = np.argmax(fac_y[fac_y.size // 2:] <= 0)
tau_y2 = np.argmax(fac_y2[fac_y2.size // 2:] <= 0)

# Gráficos das FACs
plt.figure(figsize=(10, 6))
plt.plot(fac_y, label='FAC de y')
plt.plot(fac_y2, label='FAC de y^2')
plt.axvline(tau_y, color='red', linestyle='--', label=f'Tau_y = {tau_y}')
plt.axvline(tau_y2, color='green', linestyle='--', label=f'Tau_y^2 = {tau_y2}')
plt.xlabel('Atraso')
plt.title('Funções de Autocorrelação de y e y^2')
plt.legend()
plt.grid(True)
plt.show()

# b) Verificando a correlação entre u e y
fcc_uy = correlate(u, y, mode='full')

# Gráfico da Função de Correlação Cruzada (FCC)
plt.figure(figsize=(10, 6))
plt.plot(fcc_uy, label='FCC entre u e y')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Atraso')
plt.title('Função de Correlação Cruzada entre u e y')
plt.grid(True)
plt.show()

# 2. Estimação de Parâmetros
# c) Dividindo os dados em dados de treinamento e teste
split_point = len(t) // 2
t_train, t_test = t[:split_point], t[split_point:]
u_train, u_test = u[:split_point], u[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

# d) Estimando parâmetros do modelo ARX
def arx_model(theta, u, y):
    theta1, theta2, theta3 = theta
    N = len(u)
    y_est = np.zeros(N)
    for k in range(2, N):
        y_est[k] = theta1 * y_est[k - 1] + theta2 * u[k - 1] + theta3 * y_est[k - 2]
    return y_est

def aic_loss(theta, u, y):
    y_est = arx_model(theta, u, y)
    residuals = y - y_est
    sigma2_xi = np.var(residuals)
    n_theta = len(theta)
    aic = len(u) * np.log(sigma2_xi) + 2 * n_theta
    return aic

# Inicialização dos parâmetros
initial_params = [0.1, 0.1, 0.1]  # Valores iniciais arbitrários

# Otimização dos parâmetros usando minimize do SciPy
result = minimize(aic_loss, initial_params, args=(u_train, y_train))
optimal_params = result.x

# e) Estimando y para dados de treinamento e teste
y_train_est = arx_model(optimal_params, u_train, y_train)
y_test_est = arx_model(optimal_params, u_test, y_test)

# Gráficos de comparação entre os dados reais e estimados
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(t_train, y_train, label='y (treinamento)')
plt.plot(t_train, y_train_est, label='y_est (treinamento)')
plt.xlabel('Tempo')
plt.ylabel('y')
plt.title('Dados de Treinamento')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(t_test, y_test, label='y (teste)')
plt.plot(t_test, y_test_est, label='y_est (teste)')
plt.xlabel('Tempo')
plt.ylabel('y')
plt.title('Dados de Teste')
plt.legend()

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'data_TRMS.mat'